<a href="https://colab.research.google.com/github/k-timy/Keras-GAN/blob/master/GAN_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Let's Setup the Environment first!**

We need to install Tensorflow and Keras versions 2 (the newest versions are fine, however, they should not be before 2.1)

In [1]:
# Upgrading Colab's frameworks

!pip install keras --upgrade
!pip uninstall tensorflow
!pip install tensorflow==2.1


     |████████████████████████████████| 378kB 3.4MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 3.8MB 31.7MB/s 
     |████████████████████████████████| 450kB 60.2MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
ERROR: tensorboard 2.1.0 has requirem

## **The Algorithm**

Here is the algorithm from the paper:

![alt text](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/05/Summary-of-the-Generative-Adversarial-Network-Training-Algorithm-1024x669.png)



## **Let's get into it!**

The rest of this notebook is the implementation of the Generative Adversarial Network using multi-layered neural perceptrons (MLP). First the main libraries are imported:

In [1]:


from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import mnist
import tensorflow as tf


# initial preprocessing image dimensions:
img_rows, img_cols = 28, 28
num_classes = 10

# Just to make sure the tf version is 2.1.0 (or newer)
print(tf.__version__)


2.1.0


Using TensorFlow backend.


## **Gimme the Data!**

We load the set of images including 60000 training and 10000 testing handwritten images from the dataset of MNIST.

Notice that the test set is also loaded here. However, it is not necessary for GAN to load the test set.

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if len(y_train.shape) < 2:
  # convert class vectors to binary class matrices
  # this "if" condition here, makes sure that the to_categorical function is 
  # only called once. And prevents the code from adding further dimensions
  # to the y vectors(if the code is run again during the same runtime execution)
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes) 

# Since we are implementing an MLP, we convert the 2D images of size 28x28 into
# 1D vectors of size 784 (=28x28)
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)



Taking a look at the shape of the loaded arrays of image:

In [4]:
[x_train.shape ,y_train.shape]

[(60000, 784), (60000, 10)]

The imported data needs some preprocessing. Converting the image data to float data type and normalizing them to fall in range [0,1].

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 784)
60000 train samples
10000 test samples


We define the Generator and the Discriminator classes here.
Since both of these classes are **MLPs**, the pretty much cleaner and easier way to implement the structure of them is to use the sequential model as described in the official Keras's documentation [here](https://keras.io/getting-started/sequential-model-guide/):



```
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])
```
However, I intentionally used  this approach of implementing a sequence of layers, in order to get my hands on this method of writing code as well. This method is helpful and necessary when writing custom architectures with probably several branches of computational graph.


**Note:** For the architecture and hyper-parameters I used [this](https://github.com/lyeoni/pytorch-mnist-GAN/blob/master/pytorch-mnist-GAN.ipynb) and [this](https://github.com/eriklindernoren/Keras-GAN/blob/master/acgan/acgan.py) implementations as references for implementation.



## **The two Rivals! The Generator and The Discriminator**

The following piece of code defines the MLPs of Generator and Discriminator.
More explanations in the code!

In [12]:
# Link: https://github.com/lyeoni/pytorch-mnist-GAN/blob/master/pytorch-mnist-GAN.ipynb

class Generator(tf.keras.Model):
  def __init__(self,latent_var_len, hidden_layer_len, output_size):
    """
    The Generator class.
    
    To be used in GAN class as a property. That takes a vector of latent
     variable and generates.
      * `call()`: feeds the input of size `latent_var_len` to the generative MLP
       and outputs an image in form of a vector of `output_size` dimensions. 
    
    # Arguments
        latent_var_len: The number of dimensions of the latent vector.
        hidden_layer_len: The number of dimensions of the first hidden layer.
          The other hidden layers will be twice as size of the previous hidden
          layer in dimensions.
        output_size: The number of dimensions of the vector that represents
         a generated image. This needs to be converted to a 2D array, in order
         to be visualized. For example for the MNIST dataset, this will be a
         vector with length 784. That needs to be converted to a 2D array of 
         28 * 28.
    """
    super(Generator, self).__init__()
    
    # Setting up the input layer
    self.input_layer = keras.Input(shape=(latent_var_len,),
                                   name='inp_latent_var')
    
    # Dense: layers are fully connected network (FCN)

    # BatchNormalization: layers, perform normalization on the outputs of each
    # FCN that results in faster convergence.
    
    # LeakyReLU : Provide unsaturated non-linearity so that the training speed
    # increases.
    self.dense1 = layers.Dense(hidden_layer_len,
                               name='dense_1')(self.input_layer)
    self.bo1 = layers.BatchNormalization()(self.dense1)
    self.lr1 = layers.LeakyReLU(alpha=0.2)(self.bo1)

    self.dense2 = layers.Dense(hidden_layer_len * 2,name='dense_2')(self.lr1)
    self.bo2 = layers.BatchNormalization()(self.dense2)
    self.lr2 = layers.LeakyReLU(alpha=0.2)(self.bo2)

    self.dense3 = layers.Dense(hidden_layer_len * 4,name='dense_3')(self.lr2)
    self.bo3 = layers.BatchNormalization()(self.dense3)
    self.lr3 = layers.LeakyReLU(alpha=0.2)(self.bo3)

    self.dense4 = layers.Dense(output_size, activation='tanh',
                               name='dense_4')(self.lr3)
    
    # Wrapping all the computational graph in a single object so that it can
    # be called in the __call__ function
    self.gen = tf.keras.Model(inputs=self.input_layer, outputs=self.dense4)

  def __call__(self, inputs):
    return self.gen(inputs)
  

class Discriminator(tf.keras.Model):
  def __init__(self,input_image_size, hidden_layer_len=1024):
    """
    Discriminator class.
    
    To be used in GAN class. The purpose of this model is to identify the fake
    images from the real ones.
        * `call()`: Contains the logic for loss calculation using `y_true`, `y_pred`.
    
    # Arguments
        input_image_size: The size of the image as a vector. i.e. width x height
        hidden_layer_len: The size of the first hidden layer. The size of the 
        next hidden layers will be half of their previous ones.
    """

    super(Discriminator, self).__init__()

    # Dense: layers are Fully Connected Networks (FCN).
    # LeakyReLU: as explained for the Generator class.
    # Droput: Increases the regularization of the MLP by reducing overfitting.

    # Setting up the input layer of the MLP
    self.input_layer = keras.Input(shape=(input_image_size,),name='inp_image_var')
    self.dense1 = layers.Dense(hidden_layer_len, name='dense_1')(self.input_layer)
    self.lr1 = layers.LeakyReLU(alpha=0.2)(self.dense1)
    self.do1 = layers.Dropout(rate=0.3)(self.lr1)

    self.dense2 = layers.Dense(hidden_layer_len // 2,name='dense_2')(self.do1)
    self.lr2 = layers.LeakyReLU(alpha=0.2)(self.dense2)
    self.do2 = layers.Dropout(rate=0.5)(self.lr2)
    
    self.dense3 = layers.Dense(hidden_layer_len // 4, name='dense_3')(self.do2)
    self.lr3 = layers.LeakyReLU(alpha=0.2)(self.dense3)
    self.do3 = layers.Dropout(rate=0.3)(self.lr3)
    
    self.dense4 = layers.Dense(1, activation='sigmoid', name='dense_4')(self.do3)
    
    # Wrapping up the input and output as a single model.
    self.disc = tf.keras.Model(inputs=self.input_layer, outputs=self.dense4)
  
  def __call__(self, inputs):
    return self.disc(inputs)

# Just to make sure that the code is run.
print(Generator,Discriminator)

<class '__main__.Generator'> <class '__main__.Discriminator'>


## **Let's Define Generative Adversarial Network!**

The **Generative Adversarial Network (GAN)** is defined as a class in the following code:

In [20]:
import numpy as np

# A class for storing samples of generated images
import imageio
import os
import time

class MyGAN:
  def __init__(self,image_size,img_classes, disc_hidden_layer_len,gen_hidden_layer_len,latent_var_size):
    """ My implementation of the GANs.
    
    # Arguments
        image_size: The size of the image as a vector. i.e. width x height
        img_classes: Number of image classes. (Not used in this implementation)
        disc_hidden_layer_len: The number of nodes in the first hidden layer of
        the discriminator.
        gen_hidden_layer_len: The number of nodes in the first hidden layer of 
        the generator.
        latent_var_size: The size of the latent variable vector.
    """
    super(MyGAN,self).__init__()

    # Initializing Generator and Discriminator given the values.  
    self.generator = Generator(latent_var_size,gen_hidden_layer_len,image_size)
    self.discriminator = Discriminator(image_size,hidden_layer_len=disc_hidden_layer_len)

    # Setting up some values as properties
    self.latent_var_size = latent_var_size
    self.image_classes = img_classes
    self.image_size = image_size

    # Setting up loss functions. Since there are only two classes of images in
    # This implementation of GAN (fake=0, real=1), we consider using 
    # BinaryCrossEntropy
    self.gen_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    self.disc_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

    # Setting up optimiziers for each of the MLPs
    self.gen_opt = tf.keras.optimizers.Adam(learning_rate=2e-4,beta_1=0.5)
    self.disc_opt = tf.keras.optimizers.Adam(learning_rate=2e-4,beta_1=0.5)

  def __train_generator_one_batch(self,x):        
    """
    Train the generator with one batch of input images (x). Though, only the
    size of batch is used for this training and the generator does not have
    direct access to the images of the dataset. It is only trained based on the
    gradients passed from the discriminator.
    """

    # Keeping track of the computations in a tape:
    with tf.GradientTape() as tape:

      # Drawing N samples of latent vectors with normal distribution.
      # where N is the size of the batches.
      z = tf.keras.backend.random_normal((x.shape[0],self.latent_var_size))

      # Deceiving the discriminator by assiging the class of real images
      # to fake images
      y = tf.ones(x.shape[0],1)

      # generating images
      gen_out = self.generator(z)

      # classifying the generated images
      disc_out = self.discriminator(gen_out)

      # calculating the loss of classification to be passed to the generator
      gen_loss = self.gen_loss_fn(y, disc_out)
    
    # calculating gradients of the generator weights based on the loss
    grads = tape.gradient(gen_loss, self.generator.trainable_weights)

    # updating the weights of the generator using the gradients.
    self.gen_opt.apply_gradients(zip(grads, self.generator.trainable_weights))

    # returning the loss of classification of generated samples
    return gen_loss.numpy()

  def __train_discriminator_one_batch(self,x):
    """
      Train the discriminator with one batch of real image samples (x) 
      through the adversarial process.
    """

    # Keeping track of the computations in a tape:
    with tf.GradientTape() as tape:        
      
      # train discriminator on real data
      x_real, y_real = x, tf.ones((x.shape[0],1))
      disc_real_out = self.discriminator(x_real)
      disc_real_loss = self.disc_loss_fn(y_real,disc_real_out)
    
      # train discriminator on fake data

      # drawing samples of latent variables
      z = tf.keras.backend.random_normal((x.shape[0],self.latent_var_size))
      
      # generating fake images from the latent variables given
      x_fake = self.generator(z)
      y_fake = tf.zeros((x.shape[0],1))

      # calculating loss of classification of fake images
      disc_fake_out = self.discriminator(x_fake)
      disc_fake_loss = self.disc_loss_fn(y_fake,disc_fake_out)

      # sum both losses of fake and real classifications
      disc_loss_total = disc_fake_loss + disc_real_loss
    
    # calculating the gradients of the discriminator from the total loss 
    # of classifications
    grads = tape.gradient(disc_loss_total, self.discriminator.trainable_weights)

    # updating weights of the discriminator MLP
    self.disc_opt.apply_gradients(zip(grads, self.discriminator.trainable_weights))

    # returning the discriminator loss
    return disc_loss_total.numpy()

  def train_one_batch(self,x,disc_runs=1):
    """
      Train the GAN using the algorithm described in the paper of GAN,
      given a batch of input images (x)
    """

    # train discriminator for `disc_runs` epochs. The default value is 1 and it
    # works well. However, I wrote this code to follow the algorithm
    # described in GAN paper.

    d_losses = []
    for i in range(disc_runs):
      d_losses.append(self.__train_discriminator_one_batch(x))
    
    # train the generator
    
    g_loss = self.__train_generator_one_batch(x)
    d_loss = np.asarray(d_losses).mean()

    return [d_loss, g_loss]

  def sample_images(self, epoch):
    """
      Sample 200 images from the GAN and store them as a single image file of
       20x10 tiles of small images. The `epoch` argument is only passed so
      that the saved images can be distinguished from each other.

      The function returns the name of stored image as an string. 
    """
    r, c = 20, 10
    
    # Draw r x c latent samples
    z = tf.keras.backend.random_normal((r * c,self.latent_var_size))

    # Generate images
    x_fake = self.generator(z)
    
    # Rescale images into the range of [0,1]
    gen_imgs = 0.5 * x_fake.numpy() + 0.5

    # Reshape the images into an array containing 200 2D images of size 28 x 28
    gen_imgs = gen_imgs.reshape(x_fake.shape[0],28,28)

    # placing images on a big image of size (r x 28) x (c x 28)
    canvas = np.zeros((r * 28,c * 28))
    
    # index of image on gen_imgs array
    cnt = 0
    for i in range(r):
        for j in range(c):
            # storing each image on its respective place on canvas            
            canvas[i * 28:(i+1) * 28,j * 28:(j+1) * 28] = gen_imgs[cnt,:,:]
            cnt += 1
    
    fname = 'samples_from_my_gan_epoch_{}.png'.format(epoch)
    
    # saving image file
    imageio.imwrite(fname,canvas)
    return fname
    
# Just to make sure the code is run. Some times you think you have clicked on
# the run button, but in fact you have not :D
print(MyGAN)

<class '__main__.MyGAN'>


## **MyGAN Class in Action!**

Let's see how this GAN class performs. I ran this code and in 50 epochs it works fine. At first, a dataset object is created and then during the epochs it
is trained.

In [21]:
import numpy as np

# create an instance of teh class
mygan = MyGAN(28*28,10,1024,256,100)

# Prepare the training dataset
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_dataset = train_dataset.batch(batch_size)

# For debugging purposes
break_loops = False

# Iterate over epochs.
epochs = 50
file_names = []

for epoch in range(epochs):
  
  if break_loops:
    break

  print('Start of epoch {}'.format(epoch))

  d_losses = []
  g_losses = []

  # Iterate over the batches of the dataset.
  for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    dl,gl = mygan.train_one_batch(x_batch_train)
    d_losses.append(dl)
    g_losses.append(gl)    

  # aggregate losses
  d_losses = np.asarray(d_losses)
  g_losses = np.asarray(g_losses)

  # Sample some images and store them
  if epoch % 5 == 0:
    file_names.append(mygan.sample_images(epoch))
    break
  print('epoch {} : disc: {:.4f} gen: {:.4f}'.format(epoch,d_losses.mean(),g_losses.mean()))


Start of epoch 0


## **Where the images at?**

Now that the training process is completed. Lets take a look at how the generated images actually look like! The following code, downloads the samples of multliplications of 5:

**Note:** In order to run the files.download() function of google colab properly, you might need to allow the **colab.research.google.com** website to use the 3rd party cookies on your google chrome. As explained [here](https://stackoverflow.com/questions/53581023/google-colab-file-download-failed-to-fetch-error). Otherwise, you might get some errors.


In [0]:
from google.colab import files

for f in file_names:  
  files.download(f)  

I hope this piece of code helps you in getting started with the Keras and Tensorflow library. Please let me know of your feedbacks. Thanks.